In [1]:
# 0.Importing the libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error
from datetime import datetime

In [2]:
# 1.Load the dataset
file_path = r"C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\Data\merged_tables.csv"
df = pd.read_csv(file_path)

In [3]:
# 2.Filter for the specific branch 'G Handel'
branch_name = 'C Industrie'
df = df[df['BedrijfstakkenBranchesSBI2008'].str.strip().str.lower() == branch_name.strip().lower()]

In [4]:
# 3.Verify and process the 'Year' and 'Quarter' columns for date alignment
if 'Year' not in df.columns or 'Quarter' not in df.columns:
    raise KeyError("The 'Year' and 'Quarter' columns are required in the dataset.")

In [5]:
# 4.Create a 'Date' column representing the start of each quarter using PeriodIndex.from_fields
df['Year'] = df['Year'].astype(int)
df['Quarter'] = df['Quarter'].astype(int)
df['Date'] = pd.PeriodIndex.from_fields(year=df['Year'], quarter=df['Quarter'], freq='Q').to_timestamp()

In [6]:
# 5.Drop rows where 'Date' could not be created
df = df.dropna(subset=['Date'])

# Set 'Date' as the index for time-series analysis
df.set_index('Date', inplace=True, drop=True)

# Remove duplicate index values if they exist
df = df.loc[~df.index.duplicated(keep='first')]

In [7]:
# 6.Set the frequency explicitly
try:
    df = df.asfreq('QS')  # Infer the quarterly frequency and set it explicitly
except ValueError as e:
    print(f"Warning: {e}. Frequency could not be set explicitly.")

# Print unique values in 'BedrijfstakkenBranchesSBI2008' to verify
print("Unique values in 'BedrijfstakkenBranchesSBI2008' after filtering:")
print(df['BedrijfstakkenBranchesSBI2008'].unique())

Unique values in 'BedrijfstakkenBranchesSBI2008' after filtering:
['C Industrie']


In [8]:
# 7.Define parameters for SARIMAX model
branch_name = 'C Industie'
target_column = '80072ned_Ziekteverzuimpercentage_1'
exog_columns = ['81589NED_Rechtspersonen_13', '85663NED_CaoLonenPerMaandExclBijzBeloningen_1']

In [9]:
# 8.Prepare the target and exogenous variables for training and testing
y = df[target_column]
exog = df[exog_columns]

In [10]:
# 9.Split data into training (2008-2021) and testing (2022)
train_end_year = 2021
test_year = 2022
train_df = df[df['Year'] <= train_end_year]
test_df = df[df['Year'] == test_year]

In [11]:
# 10.Prepare the target and exogenous variables for training and testing
y_train = train_df[target_column]
exog_train = train_df[exog_columns]
y_test = test_df[target_column]
exog_test = test_df[exog_columns]

In [12]:
# 11.Handle missing values by filling with the mean
y_train = y_train.fillna(y_train.mean())
exog_train = exog_train.fillna(exog_train.mean())
y_test = y_test.fillna(y_train.mean())
exog_test = exog_test.fillna(exog_train.mean())

In [13]:
# 12.Fit the SARIMAX model
print(f"\nFitting SARIMAX model for branch '{branch_name}'...")
model = sm.tsa.SARIMAX(y_train, exog=exog_train, order=(1, 1, 2), seasonal_order=(1, 0, 1, 4), enforce_stationarity=False, enforce_invertibility=False)
results = model.fit(disp=False)

# Print model summary
print(results.summary())


Fitting SARIMAX model for branch 'C Industie'...
                                       SARIMAX Results                                        
Dep. Variable:     80072ned_Ziekteverzuimpercentage_1   No. Observations:                   56
Model:                SARIMAX(1, 1, 2)x(1, 0, [1], 4)   Log Likelihood                  -1.811
Date:                                Sun, 24 Nov 2024   AIC                             19.621
Time:                                        17:09:34   BIC                             34.591
Sample:                                    01-01-2008   HQIC                            25.278
                                         - 10-01-2021                                         
Covariance Type:                                  opg                                         
                                                    coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------

c:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\uwv\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [14]:
# 13.Make predictions for the test year
y_pred_test = results.predict(start=y_test.index[0], end=y_test.index[-1], exog=exog_test)

In [15]:
# 14.Calculate MAE for Q1 and all quarters of the test year
q1_mask = test_df['Quarter'] == 1
mae_q1 = mean_absolute_error(y_test[q1_mask], y_pred_test[q1_mask])
mae_all = mean_absolute_error(y_test, y_pred_test)

print(f"\nMAE for Q1 {test_year} for branch '{branch_name}': {mae_q1:.4f}")
print(f"MAE for all quarters of {test_year} for branch '{branch_name}': {mae_all:.4f}")


MAE for Q1 2022 for branch 'C Industie': 0.6434
MAE for all quarters of 2022 for branch 'C Industie': 0.2663


In [16]:
# 15.Forecast for 2023
forecast_start = pd.Timestamp("2023-01-01")
forecast_end = pd.Timestamp("2023-10-01")
y_forecast = results.get_forecast(steps=4, exog=exog_test.iloc[:4])
forecast_mean = y_forecast.predicted_mean

In [17]:
# 16.Visualization with Combined Plot for Test, Predictions, and Forecast
fig = go.Figure()

# Add actual sick leave percentage line (test data)
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test,
    mode='lines+markers',
    name='Actual',
    line=dict(color='#0078d2', width=2)
))

# Add predictions line for test period
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_pred_test,
    mode='lines+markers',
    name='Predictions',
    line=dict(color='orange', width=2, dash='dash')
))

# Add forecast line for future period
fig.add_trace(go.Scatter(
    x=pd.date_range(start="2023-01-01", periods=4, freq='QS'),
    y=forecast_mean,
    mode='lines+markers',
    name='Forecast',
    line=dict(color='green', width=2, dash='dot')
))

# Add MAE values as text annotations
fig.add_annotation(
    xref="paper", yref="paper", x=0.00, y=1.15, showarrow=False,
    text=f"MAE for all four quarters: {mae_all:.4f}",
    font=dict(size=12, color="black")
)

fig.add_annotation(
    xref="paper", yref="paper", x=0.00, y=1.20, showarrow=False,
    text=f"MAE for next quarter (Q1): {mae_q1:.4f}",
    font=dict(size=12, color="black")
)

# Set layout and design for better visualization
fig.update_layout(
    title=f'Sick Leave Percentage - Test, Predictions, and Forecast for {branch_name.capitalize()}',
    xaxis_title='Date',
    yaxis_title='Sick Leave Percentage',
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickformat="%Y-%m",  # Year-Month format for clearer date labeling
        range=[y_test.index.min(), pd.Timestamp("2022-10-01")]
    ),
    yaxis=dict(
        showgrid=True, gridcolor='lightgrey', showline=True, linewidth=0.5, linecolor='black'
    ),
    legend=dict(
        x=0.91, y=1.5, traceorder="normal"
    ),
    font=dict(family="Roboto", size=14),
    margin=dict(l=50, r=50, t=80, b=50),
    width=1100, height=500
)

# Show Plot
fig.show()

# Optionally Save Plot as HTML
fig.write_html("C:/Users/c.hakker/Downloads/Sick_leave_predict_sarimax_g_handel.html")